In [9]:
import os
import cohere
import langchain_cohere
from pprint import pprint
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
INDEX_PATH = "data/memo_index"

In [3]:
GENERATION_PROMPT = \
"""
You are a investment analyst assistant. Your job is to generate investment reports that summarize a provided investment memo, provided in section Investment Memo. Ensure you generate your summary with structure, format, quantitative metrics, and level of detail similar to the provided example in the section Format Example. The document should provide a comprehensive analysis of the investment opportunity, including transaction details, valuation metrics, company overview, return expectations, and investment merits.

## **Document Format and Content Guidelines:**

1. **Introduction and Investment Summary:**
   - Start with a clear statement of the investment proposal, including the target investment amount and the nature of the investment opportunity (e.g., co-investment, acquisition).
   - Identify key stakeholders and co-investors, mentioning their roles and the target company’s industry and business model.

2. **Transaction and Valuation Overview:**
   - Provide a detailed overview of the transaction, including the total enterprise value (TEV), purchase price, and multiples (e.g., EV/EBITDA).
   - Compare the valuation metrics with relevant market or precedent transaction comps to contextualize the deal’s attractiveness.
   - Explain the ownership structure post-transaction and any notable changes in control or governance.
   - Outline financing sources, specifying debt and equity contributions and relevant leverage ratios.
   - Highlight any significant deviations from previous transaction negotiations or market expectations.

3. **Company Overview:**
   - Present a detailed company profile, including its founding date, primary business operations, and key market segments.
   - Discuss the company’s financial performance, focusing on key metrics like revenue, EBITDA, and margins.
   - Describe the company’s customer base, market position, and scale of operations, highlighting any unique selling points or competitive advantages.
   - Break down the company’s business lines, specifying primary revenue drivers and any secondary, non-core offerings.

4. **Return Overview:**
   - Provide a return analysis, including base case assumptions for key metrics such as EBITDA multiples, revenue growth rates, margin expansion, and exit scenarios.
   - Include projected IRR (Internal Rate of Return) and MOIC (Multiple on Invested Capital) figures, both gross and net, for clients.
   - Discuss the primary value creation drivers, including organic growth, M&A opportunities, and any expected multiple expansions.
   - Highlight potential exit routes and the financial implications of different exit scenarios.

5. **Investment Merits:**
   - Identify and elaborate on key investment merits, such as market growth potential, competitive positioning, customer satisfaction, and historical performance.
   - Highlight the company’s technological strengths, proprietary assets, and ability to innovate within its market.
   - Discuss customer relationships, retention rates, and the broader value proposition offered by the company’s solutions.
   - Include information about the management team and their track record, emphasizing their experience and previous success in driving growth.

6. **Risk Factors (Optional Section):**
   - Outline any potential risks or challenges associated with the investment, including market, operational, financial, or regulatory risks.

#### **Formatting and Reporting Metrics:**
- Use concise yet comprehensive paragraphs to convey key information.
- Incorporate quantitative metrics and financial ratios prominently (e.g., TEV, EBITDA multiples, IRR, MOIC).
- Use bullet points for listing investment merits or other key points to enhance readability.
- Include comparative data where relevant to provide context (e.g., market comps, historical performance vs. peers).
- Ensure the tone remains formal and analytical, with a focus on presenting data-driven insights.

#### **Verbosity and Style:**
- Maintain a professional tone with a focus on precision and clarity.
- Aim for a balance between detailed analysis and concise summaries, ensuring that key points are highlighted without excessive verbosity.
- Use industry-specific terminology accurately to convey expertise and understanding of the subject matter.

These guidelines will help generate consistent, informative, and professionally structured investment reports that meet the analytical depth and format exemplified in the provided sample.

### **Format Example**

We seek your final approval to invest a minimum of $10 million in a co-investment opportunity alongside KKR and Veritas (co-lead) to acquire Cotiviti, a leading provider of payment integrity technology and services to healthcare payors in the U.S., helping them improve payment accuracy before and after claims are paid by seeking to identify inaccurate or fraudulent claims. 

The transaction values Cotiviti at a TEV of $10.5 billion (pre fees and expenses), or ~12.4x 2023E EBITDA. This compares favorably versus precedent transaction comps that have traded at an average LTM / NTM Adj EBITDA multiple of ~20x / 17x. Veritas, which currently wholly owns the company, will be selling 50% of the stake to KKR and retain the remaining 50% by selling it to its latest fund, Veritas Capital Fund VIII. Post transaction, KKR and Veritas would share equal ownership and joint control of Cotiviti.

KKR's base case assumes multiple expansion which implies a 2.8x/2.4x gross/net MOIC to clients and 23.2%/19.0% gross/net IRR to clients. 

#### Transaction and Valuation Overview:

KKR has agreed to buy a 50% stake in Veritas owned Cotiviti valuing the company at a TEV of $10.5 billion (pre fees and expenses), or ~12.4x 2023E EBITDA of $845 million. Veritas would be selling the remaining 50% of its stake in Cotiviti to its latest fund (“Veritas Capital Fund VIII”) from its earlier funds. Pro-forma, KKR and Veritas will share equal equity ownership in Cotiviti at 50% each and exercise joint control over the company with 3 board seats each among other control provisions. The PF TEV of the company post the transaction would be $10.8 billion. Expected financing includes $5 billion in debt (detaching at ~5.7x EBITDA) and ~$6 billion of common equity with ~$3 billion coming from KKR Funds and co-investors, and ~$3 billion from Veritas Capital Fund VIII and co-investors. The post transaction leverage detachment of 5.7x comes in lower than the 7.5x implied in the initial deal talks with Carlyle in early 2023, which subsequently fell through. Cotiviti’s implied entry valuation compares favorably versus precedent transaction comps that have traded at an average LTM / NTM Adj EBITDA multiple of ~20x / 17x.

KKR’s deal team is led by Max Lin (Partner) and includes Hunter Craig (Managing Director), and Alex Ward (Principal) who have together been involved in a number of successful healthcare investments at KKR. KKR has generated an average gross IRR / MOIC of 17.4% / 2.4x in healthcare PE in the Americas. Veritas has generated an average gross IRR / MOIC of 35.0% / 3.1x in the healthcare PE. For realized/partially realized investments in healthcare it had generated an average gross IRR / MOIC of 51.0% / 5.1x.

#### Company Overview:

Founded in 2014, Cotiviti is a leading provider of payment integrity technology and services to healthcare payors, helping them improve payment accuracy before and after claims are paid by seeking to identify inaccurate or fraudulent claims. These activities can generate material savings for the payors, who in turn pay Cotiviti a percentage of the savings as revenue. The business currently generates $1.7 billion in revenues and $845 million in EBITDA translating to ~50% margins. The company works with more than 300 clients, including all of the top 25 plans in the United States, to deliver higher-performing payment accuracy, risk adjustment, quality improvement, network and clinical analytics, and consumer engagement programs. It analyzes over $1 trillion of annual healthcare spend and generates ~$9 billion of annual cost savings for customers. Payment integrity is a mission critical function for health insurers and represents a $5-7 billion total addressable market expected to grow 5-7% over the next five years. Cotiviti’s served market is expected to outpace the overall market given the company’s attractive portfolio mix indexed to higher growth pre-pay solutions.

Cotiviti’s business could be broadly divided into the following categories based on the services offered by the company:

Payment Integrity (“PI”): This is the primary service offered by the company and accounts for ~80% of the revenue. It includes reviewing claims for payment accuracy in pre-payment (prospective) & post-payment (retrospective). Cotiviti offers a full suite of pre-pay and post-pay payment integrity solutions.
Other: Remaining ~20% revenue comes from several non-core offerings, including Risk Adjustment (seek to ensure accurate payment from government for Medicare Advantage plans), Performance Analytics & Quality (help payors improve clinical quality scores), Retail (payment integrity for retailers). These offerings present additional growth vectors for the company that has traditionally focused on payment integrity solutions.

#### Return Overview:

Petiole base case assumes no multiple uplift with a 11.9x NTM EBITDA exit multiple representing a gross/net to clients MOIC of 2.4x and 2.1x, and a gross/net to clients IRR of 18.9% and 15.1%. KKR's base case assumes an exit multiple of 14.0x NTM EBITDA (vs entry multiple of 11.9x implied NTM EBITDA), 7.6% organic revenue CAGR (vs. 5.6% 2019-2023 CAGR), 10.3% revenue growth pro forma for M&A, organic EBITDA margin expansion to 54.8% (from current 51%), and an exit in 2028. The returns imply a gross/net IRR to clients of 23.2% and 19.0%, and a gross/net to clients MOIC of 2.8x and 2.4x. Exit route would be through a financial or strategic sale given historic interest from both sides. The interest coverage ratio remains above 2x in the base case with comfort around the ability to cut down on growth-oriented capex and M&A if necessary.

KKR's base case value creation drivers are approximately 2/3 from revenue growth and a 1/3 from multiple expansion. 3/4 of revenue growth will be generated organically with the remaining 1/4 coming from M&A. Main drivers to revenue growth will come through upselling and cross selling existing and new lines of business, supported by product innovation through R&D initiatives and add-on M&A derived through a proven acquisition playbook.

#### Investment Merits:

- Operates as a leading provider in a large and growing market: Payment integrity is a $5-7 billion market poised to grow at 5-7% underpinned by secular growth drivers such as claims volumes, medical inflation, and increasing complexity in the healthcare system necessitating continued innovation and customization of payment policy. Bain projects a served market growth of 6-8%, and Cotiviti’s implied market growth of 7- 10%. The company's strong product offering in pre-pay make it well-positioned to benefit from the segment’s differential growth rate of 9-11%.
- Offers high quality solutions with positive customer feedback: Cotiviti receives exceptionally positive customer feedback, with 9/10 customer satisfaction and differentiated customer value proposition in three of four segments. In the core Payment Policy Management (PPM) solution, Cotiviti outperforms Optum by 17% in value proposition, with net promoter score (NPS) of 80% vs. 33% for Optum.
- Longstanding customer relationships with a compelling value proposition: The company offers a compelling value proposition for health insurers by generating ~$9 billion in annual cost savings representing ~7% EPS contribution for payor customers, driven by a proprietary library of 25mm edits. Cotiviti has a gross customer retention rate of 99%.
Virtuous cycle of innovation underpinned by proprietary edits and longitudinal patient database: Cotiviti has the component parts – longitudinal proprietary claims data + analytics team + software substrate – to successfully execute on R&D initiatives and drive increased efficiencies through AI/ML. Since 2019, Cotiviti has launched 18 new solutions and unlocked ~$2 billion of SAM.
- Attractive growth algorithm with the potential to drive sustainable top-line growth: Cotiviti has exhibited historical payment integrity revenue CAGR of 7% (LTM Dec 2019 - Jun 2023) driven by mix of existing clients (4%) and new logos (3%). Going forward, a multi-billion revenue opportunity is supported by multiple growth levers, including medical inflation, adoption of existing edits, upsell of new edits, cross-sell of new solutions, and penetration of new customers.
- Multiple value creation levers to further accelerate growth with experienced management team to execute: The sponsor has identified several growth opportunities, including investments in sales and marketing, R&D, and risk adjustment optimization. Additionally, a historical track record of successful integration of acquisitions and exposure to multiple attractive areas present Cotiviti with an opportunity to grow further through M&A in a fragmented industry.

## Investment Memo
{document}
"""


In [4]:
DOCUMENT = \
"""
Project
Hideaway
Co-Investment
Presentation
January 2024
Privileged & Confidential
CONFIDENTIAL | 2
Disclaimer
This Co-investor Presentation (this “Presentation”) includes proprietary, sensitive and confidential information regarding Alteryx, Inc. and its direct and indirect subsidiaries (the “Company”) which has not previously been
publicly disclosed. By your receipt of this Presentation, you acknowledge and agree to maintain the confidentiality of such information and this Presentation and to not reproduce or disclose it to any other person or use it
for any purpose other than as may be agreed to in writing by the Company, and Clearlake Capital Group, L.P. (“Clearlake”) and Insight Venture Management, LLC (“Insight”) (together with their affiliates, the “Sponsors”).
Without limiting the foregoing, this Presentation and any other information being provided to the recipient, whether in writing or orally, is subject to any and all confidentiality restrictions, duties, and obligations applicable to
the recipient. In addition, by receiving this Presentation, the recipient agrees to use this Presentation and the information contained herein in accordance with its compliance policies, contractual obligations and applicable
law, including applicable foreign, United States federal and state securities laws.
THIS IS NOT AN OFFER TO SELL, OR A SOLICITATION OF ANY OFFER TO BUY ANY SECURITIES.
THIS IS A SUMMARY FOR INFORMATIONAL PURPOSES ONLY AND DOES NOT CONSTITUTE AN OFFER TO SELL, OR A SOLICITATION OF ANY OFFER TO BUY, ANY SECURITIES. ANY SUCH OFFERING
WILL BE MADE ONLY BY MEANS OF AN OFFERING DOCUMENT CONTAINING IMPORTANT INFORMATION ABOUT THE INVESTMENT OBJECTIVE, TERMS AND CONDITIONS OF AN INVESTMENT IN A FUND
AND ALSO CONTAINS TAX INFORMATION AND RISK DISCLOSURES THAT ARE IMPORTANT TO ANY INVESTMENT DECISION REGARDING A FUND, AND ONLY IN THOSE JURISDICTIONS PERMITTED BY
LAW. RECIPIENTS SHOULD NOT CONSTRUE THE CONTENTS OF THIS PRESENTATION AS LEGAL, TAX, INVESTMENT OR OTHER ADVICE. NO PERSON HAS BEEN AUTHORIZED TO MAKE ANY STATEMENT
CONCERNING A FUND OTHER THAN AS SET FORTH IN THE OFFERING DOCUMENT AND ANY SUCH STATEMENTS, IF MADE, MAY NOT BE RELIED UPON.
THE LIMITED PARTNERSHIP INTERESTS IN ANY SECURITIES RELATED HERETO WILL NOT BE REGISTERED UNDER THE SECURITIES ACT OF 1933, AS AMENDED, OR THE INVESTMENT COMPANY ACT
OF 1940, AS AMENDED, OR REGISTERED WITH, OR APPROVED BY, ANY STATE SECURITIES OR BLUE SKY ADMINISTRATOR OR ANY OTHER REGULATORY AUTHORITY. THE INTERESTS IN ANY FUND
DESCRIBED HEREIN HAVE NOT BEEN RECOMMENDED BY ANY FEDERAL OR STATE SECURITIES COMMISSION OR REGULATORY AUTHORITY. FURTHERMORE, THE FOREGOING AUTHORITIES HAVE
NOT CONFIRMED THE ACCURACY OR DETERMINED THE ADEQUACY OF ANY DISCLOSURE CONCERNING ANY FUND. ANY REPRESENTATION TO THE CONTRARY IS A CRIMINAL OFFENSE.
This Presentation and the oral remarks made in connection herewith may contain forward looking information, including, without limitation, financial projections and other estimates with respect to the anticipated
performance of the Company (the “Projections”). The Projections include statements regarding the intent, belief or current expectations of the Company and members of its management team, as well as the assumptions
on which such statements are based, and generally are identified by the use of words such as “may,” “ will,” “ seeks,”“ anticipates,” “ estimates,” “expects,” “plans,” “intends,” or similar expressions. The Projections have
been prepared in good faith based upon assumptions that the Company believes to be reasonable as of the date on the cover of the Presentation, it being understood that the Projections are as to future events and are not
to be viewed as facts, that the Projections are subject to significant uncertainties and contingencies, many of which are beyond the Company’s control, that no assurance can be given that any particular Projections will be
realized and that actual results during the period or periods covered by any such Projections may differ significantly from the projected results and such differences may be material. In addition, the Projections were not
prepared with a view to public disclosure or compliance with the guidelines established by the American Institute of Certified Public Accountants or US generally accepted accounting principles (“GAAP”) or any other
applicable accounting principles. Past performance is not indicative of future results and there can be no assurance that any of the trends described herein or any other trends will continue or materialize. There can be no
assurance that the transaction will be consummated or, if consummated, will be consummated on the terms described herein. Forward looking statements contained herein are only estimates of future results or events that
are based upon assumptions made at the time such statements were developed or made.
You acknowledge and agree that this Presentation contains and is based on information from third party sources (including the Company and the Sponsors) and it is provided to you for informational purposes. In addition,
acceptance of this Presentation further constitutes your acknowledgment and agreement that the Company, the Sponsors, the Company’s and the Sponsors’ respective direct and indirect equity holders and the Company’s
and the Sponsors’ and respective affiliates and its and their respective partners, members, equity holders, controlling persons, directors, officers, employees, agents, advisors and other representatives (collectively, the
“Representatives”) bear no responsibility (and shall not be liable) to you for the accuracy or completeness (or lack thereof) of the information in the Presentation, no representation regarding the Presentation is made by the
Company, the Sponsors or any of their respective affiliates or Representatives to you, none of the Company, the Sponsors, the Company’s and the Sponsors’ respective direct and indirect equity holders or their respective
affiliates and Representatives shall have any obligation to update or supplement the Presentation or otherwise provide additional information and the Company, the Sponsors, the Company’s and the Sponsors’ respective
direct and indirect equity holders and their respective affiliates and Representatives shall not have any liability related to the use of the Presentation.
This Presentation includes pro forma adjusted EBITDA and other non GAAP financial measures. The non GAAP measures provided herein may not be directly comparable to similar measures used by other companies in
the Company’s industry, as other companies may define such measures differently. The non GAAP measures presented herein are not measurements of financial performance under GAAP, and should not be considered
as alternatives to, and should only be considered together with, the Company’s financial results in accordance with GAAP. The Company does not consider these non GAAP financial measures to be a substitute for, or
superior to, the information provided by GAAP financial results. This Presentation also contains estimates and information concerning the Company’s industry, including market position, market size, and growth rates of the
markets in which the Company participates and are based on industry publications and reports. This Presentation and the information contained herein involves a number of assumptions and limitations, and you are
cautioned not to give undue weight to these estimates. The Company and the Sponsors have not independently verified the accuracy or completeness of the data contained in these industry publications and reports. The
industry in which the Company operates is subject to a high degree of uncertainty and risk due to variety of factors. These and other factors could cause results to differ materially from those expressed in these publications
and reports.
CONFIDENTIAL | 3
Disclaimer
This Presentation has been prepared to assist interested parties in making their own evaluation of the Company and does not purport to be all inclusive or to contain all of the information or a discussion of all of the risks
that a prospective participant may consider material or desirable in making its decision to invest in the Company. You should take such steps as you deem necessary to assure that you have the information you consider
material or desirable in making your decision to invest in the Company and should perform your own independent investigation and analysis of the Company and its affiliates, including by supplementing the information
contained this Presentation with other evaluation materials and seeking your own legal, accounting and other relevant professional advice. The information contained herein may not be relied on in any manner as legal, tax,
or investment advice. The information and data contained herein are not a substitute for your independent evaluation and analysis and should not be considered as a recommendation by the Company, the Sponsors or any
of their respective affiliates or Representatives that you enter into a potential transaction.
This Disclaimer constitutes a binding agreement between you and the Sponsors for the benefit of the Company, the Sponsors, and their respective affiliates and Representatives with respect to this Presentation, any other
information regarding the Company or the Sponsors furnished or communicated to you by or on behalf of the Company or the Sponsors in connection with the potential transaction (whether prepared or communicated,
orally or in writing, by any Sponsor, the Company or any of their respective Representatives (the “Evaluation Material”) and any memoranda, notes or other documents and analyses developed by you using any information
specified under the definition of Evaluation Material (collectively “Internal Evaluation Material”) and supersedes all prior understandings and agreements relating thereto. The terms and conditions of this Disclaimer shall
apply until such time, if any, that you become a party to the definitive agreements regarding a potential transaction, and thereafter the provisions of such definitive agreements relating to confidentiality shall govern. If you
do not enter into any definitive agreements, the application of this Disclaimer shall terminate with respect to all Evaluation Material and Internal Evaluation Material on the later of (a) the date falling two years after the date
of this Presentation, and (b) to the extent that you are a party to any other agreement or duty creating obligations pertaining to the Evaluation Material, the date that such obligations terminate.
All securities investments risk the loss of capital. No guarantee or representation is made that a partnership (or any transaction or investment referenced in this Presentation) will achieve its investment objective. An
investment in a partnership is speculative and involves certain risk factors which recipients should consider before investing. Recipients should be able to afford a complete loss of all capital invested in a partnership.
Trademarks and trade names may be used in this Presentation to refer to either the entities claiming the marks and/or names or their products and are the property of their respective owners. Insight disclaims proprietary
interest in the marks and names of others.
METHODOLOGY FOR PROJECTED PERFORMANCE OF ALTERYX CO-INVESTMENT OPPORTUNITY
To derive the projected “net multiple on invested capital” (or “Net MOIC”) for the proposed co-investment opportunity in Alteryx (the “Alteryx Co-Investment Opportunity”), the projected gross multiple of such Co-Investment
Opportunity is reduced by an estimated expense ratio. Because investors in the Alteryx Co-Investment Opportunity will not pay management fees or carried interest, the net performance included in this Presentation does
not reflect the deduction of such fees or carried interest. There is a 3% cap on expenses that can be charged to investors in the Alteryx Co-Investment Opportunity.
The net multiple on invested capital was calculated by applying a 3% expense ratio to the projected gross multiple. Because this is the highest expense ratio that can be charged to investors in the Alteryx Co-Investment
Opportunity, Clearlake and Insight believe this is a reasonable methodology to estimate the projected net multiple but is not the actual net multiple with respect to any investment in Alteryx.
The projected “net IRR” of the Alteryx Co-Investment Opportunity is calculated by reducing the projected cash inflow from such Co-Investment Opportunity by the same 3%. Clearlake and Insight believe this will yield a
projected net IRR consistent with how the projected net multiple is calculated. This also represents what Clearlake and Insight believe to be a reasonable methodology to estimate the projected net IRR based on the highest
expense ratio that can be charged to investors in the Alteryx Co-Investment Opportunity, but any such projected information is not the actual net IRR with respect to any investment in Alteryx.
Because the projected net performance information for the Alteryx Co-Investment Opportunity included in this Presentation relies upon the application of assumptions that were not experienced by individual investors that
are indirectly invested in Alteryx through a Clearlake or Insight Private Equity Fund, the projected net performance information in this Presentation with respect to the Alteryx Co-Investment Opportunity, like all projected
information or targets, is for illustrative purposes only, and the actual net investment performance an investor indirectly invested in Alteryx through Clearlake or Insight may vary significantly and may be materially different.
In addition, there are other reasonable methodologies or assumptions Clearlake and Insight could have used in calculating the projected net performance of the Alteryx Co-Investment Opportunity, which may lead to higher
expense ratios or more accurate net performance figures than those presented herein.
The performance information for the Alteryx Co-Investment Opportunity included in this Presentation is projected performance that is hypothetical. Investors should consider the limits of hypothetical performance when
making an investment decision. The projected performance has been prepared based on various assumptions made by Clearlake and Insight, which are disclosed above. Clearlake and Insight believe these assumptions to
be reasonable; however, different assumptions would result in different projected performance results, which could be higher or lower than the projected performance shown, and could be materially lower. Actual
performance results likely will differ from the projected performance shown, and may be higher or lower than the projected performance shown, and could be materially lower.
CONFIDENTIAL | 4
Executive Summary
Table of Contents
Company Overview
1
Investment Highlights
Financial Overview
2
3
4
Project
Hideaway
Executive Summary
CONFIDENTIAL | 6
Sponsor Overview
C
l
e
a
r
l
a
k
e
I
n
s
i
g
h
t
P
a
r
t
n
e
r
s
h
i
p
▪ Clearlake Capital Group, L.P. is an investment firm
founded in 2006 operating integrated businesses across
private equity, credit and other related strategies
▪ The firm takes a sector-focused approach, partnering
with experienced management teams by providing long
term capital to dynamic businesses
▪ Targets investments in software & technology-enabled
services, industrials and consumer
▪ Over $70 billion of assets under management
▪ Senior investment professionals have led or co-led over
300 investments
▪ Proprietary operational improvement approach: O.P.S.
®
− Operations
− People
− Strategy
Sampling of Tech Investments Sampling of Tech Investments
Select Partnerships
▪ Founded in 1995, Insight Partners is a private equity firm
investing in growth-stage technology and software
companies
▪ Globally diversified portfolio of 500+ active investments
across stages, deal types and verticals
▪ Over $75 billion of assets under management
▪ Currently investing out of its twelfth main fund with $20
billion of LP capital commitments
▪ Portfolio companies benefit from Insight Onsite, an in-
house team of technology experts that work alongside
management teams to improve and scale various core
functions
Note: The samplings and select partnerships presented herein are for illustrative purposes. Please see Clearlake and Insight’s respective websites for a complete list of investments.
INSIGHT
CLEARLAKE PARTNERS
Insight
Clearlake
CLEARLAKE
INSIGHT
PARTNERS
Partnership
Constant Contact 2| BeyondTrust Confluence
DAMCS
ARMIS.
BESTPASS
CoreLogic*
digicert® ConvergeOne cornerstone APPRISS'
Knowledge for
good."
ivanti lytx MetricStream E/ eagleview" Bamboo
DISTROKID Dotmatics
Kaseya
Health
OnShift PERFORCE
netdocuments* &PDI @ inhabit®
Diligent
inovalon®
newfold provation : symplr ZYWAVE pprecisely VEEam Recorded
IH
Future'
digita
CONFIDENTIAL | 7
Transaction Summary
▪ Clearlake Capital Group, L.P. (“Clearlake”) and Insight Partners (“Insight”) are pleased to
present a potential equity co-investment opportunity supporting a take-private of Alteryx, Inc.
(“Alteryx”, “Hideaway”, or the “Company”)
o Qatalyst was retained by Alteryx in May 2023 to run a strategic alternatives process that resulted
in Qatalyst reaching out to potential acquirors of the Company
o Clearlake has actively tracked Alteryx over the last ~5 years, and Insight is currently an investor
in the Company’s public equity, having invested in Alteryx in September 2014 (pre-IPO). Insight
currently has one BoD seat and a passive ownership stake of ~1.3%
▪ Alteryx is a data prep and analytics tool for non-technical users. Its platform allows users to
connect and clean data from cloud applications, data warehouses, spreadsheets and other
sources, enabling them to perform analytics and predictive modeling to turn data into insights
o Alteryx is nearing $1 billion of annual recurring revenue (ARR) with a runway for growth given
positive secular tailwinds and a growing total addressable market (TAM) of ~$39 billion(1)
o The Company has a customer base that includes 49% of the Global 2000 (2), which Clearlake
and Insight believe is driven by the Company’s brand recognition, product portfolio, and 20+ year
history of helping its customers improve their businesses
▪ The Company is expected to generate FY2023E Revenue and PF Adj. Cash EBITDA of $958
million and $364 million (38% Cash EBITDA margin)(3), respectively
(1) Boston Consulting Group market study, June 2023; (2) Alteryx Investor Presentation, November 2023; (3) Management expectations. Includes $218M of PF cost takeout based on Clearlake and Insight cost plan.
Note: Statements made herein may represent opinions and beliefs of Insight, Clearlake and Alteryx and should not be relied upon as a promise or representation as to past or future performance. Please see Disclaimers for further information regarding
projections and forward looking statement.
CONFIDENTIAL | 8
Transaction Sources and Uses
Transaction represents fully burdened TEV multiples of 13.6x 2023E PF Adj. Cash EBITDA less
CapSW and 4.9x 2023E ARR
Sources $M % Uses $M %
New Unitranche Term Loan $1,800 38.8% Equity Purchase Price (1) $3,556 76.6%
Total Equity Account 2,844 61.2% Existing Net Debt 568 12.2%
Estimated Transaction and Financing Fees 220 4.7%
Cash to B/S 300 6.5%
Total Sources $4,644 100.0% Total Uses $4,644 100.0%
Memo:
Clearlake Funds and Co-Invest Equity $1,706 36.7%
Insight Partners Funds and Co-Invest Equity $1,137 24.5%
(1) Equity Purchase Price based on estimated share count as of 03/31/24E (includes basic shares outstanding, in-the-money options, and single trigger acceleration RSUs / PSUs).
Note: Please see Disclaimers herein for important information on estimates, projections and forward looking statements. Certain information contained herein has been obtained from third parties, including Company
management. While such information is believed to be reliable, such information may be based on assumptions that later prove to be invalid or incorrect and is not provided with a view to providing investment advice with
respect to any security, or making any claim as to the past, current or future performance thereof.
CONFIDENTIAL | 9
Investment Highlights
1 Leading Player in Large and Growing Market
▪ ~$39B TAM growing at ~12% per annum(1) as Alteryx is a leader in the data prep and
blend market for non-technical users(2)
2 Revenue at Scale
▪ Approaching $1B of ARR growing mid-teens with historical loss-only retention of 95%+
and 2023E net retention of ~112% with limited customer concentration
3 Core Software Solutions
▪ Core data prep-and-blend solution embedded in operational workflows and utilized
across functional groups with 80+ data connectors to all critical data sources
4 Diversified Customer Base
▪ Product leadership (20+ year incumbent) has resulted in a customer base of 49% of
the Global 2000(3) with 8,000+ total customers
5 Opportunity to Expand Profitability
▪ Software benchmarking points to opportunity to optimize costs leveraging Clearlake’s
O.P.S.® framework and Insight’s operational best practices – detailed cost plan ready
to be implemented on day one
6 Platform for M&A
▪ Opportunity to leverage combined Clearlake and Insight resources to pursue accretive
and complementary targets
(1) Boston Consulting Group market study, June 2023; (2) Leading player in data preparation and analytics per Bain market study, November 2023; (3) Alteryx Investor Presentation, November 2023.
Note: Past performance is not indicative of future results and there can be no assurance that any of the trends described above or any other trends will continue or materialize, and there can be no assurance that any assumption forming a part of the estimates or
projections contained herein will continue to be true. Please see Disclaimers herein for important information on estimates, projections and forward looking statements.
alteryx
Project
Hideaway
Company Overview
CONFIDENTIAL | 11
A Brief History of Alteryx
1997
Alteryx was founded
As SRC, Inc.
2010
SRC was renamed
Alteryx
2006
Insight Partners first
connected with
Dean Stoecker
2014
Alteryx Server
(Private Cloud) was
released
Sept. 2014
Insight Partners
invested in Alteryx
2017
Acquisition of
Semanta, the
foundation for
Alteryx Connect
2017
Alteryx goes public
(NYSE: AYX)
2017
Acquisition of Yhat,
the foundation for
Alteryx Promote
2019
Acquisition of
ClearStory Data
2019
Acquisition of
Feature Labs
2021
Acquisition of Hyper
Anna
2021
Acquisition of
Lore IO
2022
Launch of Alteryx
Cloud
2022
Acquisition of
Trifacta
CONFIDENTIAL | 12
Alteryx by the Numbers
~14% ARR Growth
~$948M ARR ~112% Net Retention
8,000+ Customers
~90% Gross Margin
Customers Include
~49% of the
Global 2000
160+ Customers w/
$1M+ of ARR
~$39B TAM $100K+ Average ARR
per Customer
All financial metrics are for FY2023E, November 2023.
Note: Past performance is not indicative of future results and there can be no assurance that any of the trends described above or any other trends will continue or materialize.
=
CONFIDENTIAL | 13
The Alteryx Value Proposition
The Challenge: The Solution:
Organizations increasingly rely on data for critical
operations, but often lack the talent or resources to turn
data into actionable insights
Alteryx enables the everyday business analyst to be a data
scientist – saving valuable time and powering invaluable
workflows embedded in day-to-day operations
Data from Disparate Platforms that Rarely Integrate Easily
Hours Lost and Critical Errors Made in Spreadsheets
Coders are Expensive, and Don’t Fully Solve the Problem
“We generated over $10,000,000 in value, saved 40,000 hours
of manual work, and retired 500 spreadsheets”
“The benefit of using Alteryx isn’t just the fact that we can do things
10 times faster. It allows us to do things that we never dreamed
of before”
Coders often lack context
behind the underlying data,
which is often industry or
company specific, creating
costly back-and-forth between
functional groups and IT teams
Easy to Use
Advanced Capabilities
Data science solution democratizes analytics (new users can
learn the basics in a matter of hours)
Ability to process and combine gigabytes of data from 80+
sources(1)
Automate Entire Analytics Processes
Memorizes user-created workflows and can rerun the same
analyses on new sets of data. For instance, Alteryx can
automatically create a sales report that analyzes in-depth
performance by rep every time that new data becomes
available, saving users countless hours
Source: Boston Consulting Group market study, June 2023. The views of other customers may differ.
O Google salesforce
X
ORACLE Analytics
O
AAnaplan aws Marketo' snowflake
#REFI MREF NREF
#REFI #REFI
Explorer. EXE X
UREFI #REF #REF #REFI #REFI ×
HREF! #REF
F
#REF! #RFF! #REF The
file
name is too
long.
BREF! #REF! #REF! #REF! #REF!
WREF! MREF!
F
MREF! WREF!
#REF!
F
HREF! #REF! HREF AREF! HREF! OK
#REF! MREF!
#REFI #REFI
#REFI
#REFI #REFI #REFI
#REFI
BREF
F #REFI #REFI #REF
#REF! F
#REF! #REF #REF
An error occurred while loading the data. #REF! HREF!
#REF!
#REF!
HREF!
WREF!
WREF!
MREF!
Show Details HREF! HREF! HREF! HREF!
-----
..
CONFIDENTIAL | 14
Platform Overview
Description / Offerings
▪ On-prem data preparation tool
primarily for joining and cleaning
across multiple sources
▪ Platform for sharing, automating,
and governing workflows across
an organization within the
customers own on-prem or cloud
environment
▪ New solution that brings on-prem
Designer capabilities to the
cloud; includes sharing and
automation capabilities (without
the need for Server) and expands
processing capabilities beyond
Designer
▪ Supplementary solutions
enhancing insights and
increasing analytical functionality
(e.g., Location Intelligence, Auto
Insights, etc.)
Core Product Families Platform Features
Designer
Server
Designer Cloud
Add-Ons
No programming required
Provides data analytics to all users
Community allows users to gain insights from each
other
Process data within lower memory environments
Connectivity with R and Python for analytical
models
Open and modular core enabling additional
customization
Source: Boston Consulting Group market study, June 2023.
.....
.....
CONFIDENTIAL | 15
Alteryx Core Strengths in the Data Value Chain
Data Sources Data Preparation BI / Data Analytics Outputs
Local Spreadsheets
Cloud Spreadsheets
Cloud Databases
Cloud Data Lakes /
Data Warehouses
Data Preparation
Data Cleaning &
Selection
Data Aggregation
Visualization Prep
Workflow Automation
Dynamic Dashboards /
Reporting
Basic Formulas / Data
Summarization
Advanced Analytics
Local / Cloud
Spreadsheets
Data Visualization
Outputs
Cloud Databases
AI / ML Workloads
Geospatial Analysis
Path from Data to Decision Making
Majority of the time spent by
analysts(1)
Alteryx Core Strengths
Source: Boston Consulting Group market study, June 2023; (1) GLG expert network calls.
CONFIDENTIAL | 16
Product Usage Evolution
How the
Data is
Accessed
• Data is stored and
opened in the native .xls
formats and typically
does not require
connection to an
external data source
• Connect to virtually any
data source (local, cloud,
or on-prem) and open the
file format within Alteryx
Use Case
• Typically, ad-hoc
analyses and
preparation of smaller
datasets due to size
limitations
• Prepare larger datasets for
ad-hoc analyses
• Schedule automated
workflows for regular
analyses and reporting
How the
Data is
Outputted
• Typically saved within
the Excel format and
ecosystem
• Outputted to local device,
cloud, on-prem, etc. in
different formats
• Published in dashboards,
automated reports, etc.
“We use Hideaway for a lot of what Excel can't do...when the data
size is too large and when we want to set up weekly repeated
workflows”
“Our use of Hideaway has changed significantly.
Initially we used it for parsing out tax statements,
retrieving K1s, and storing that data...today we parse
out data, have set up automated alerting, and can
ingest hundreds of files a day”
20%
40%
60%
80%
Jan-20 Nov-20 Sep-21 Jul-22 May-23
% Complex (1) % Automated
Increasingly More Complex(1) and Automated Workflows
Differentiation from Excel
Legacy Approach to Data The Redefined Approach % Share of Complex(1) and Automated Workflows
~70%
~65%
Source: Bain and Boston Consulting Group market studies. The views of other customers may differ.
(1) Defined as workflows utilizing 30+ platform tools (e.g., predictive analytics, spatial data, macros, etc.).
X alteryx
-
CONFIDENTIAL | 17
34%
28%
14% 14%
5% 5%
1 2 3 4 5 6
Number of Data Input Sources per Workflow(2) Number of Data Output Sources per Workflow(2)
8%
43%
24%
17%
4% 3%
1 2 3 4 5 6
Alteryx is Positioned for Complex Data Needs
The Company is positioned to power use cases involving multiple data inputs and outputs due to
stack-agnostic capabilities and 80+ connectors(1)
66% of uses involve data inputs
from multiple applications
92% of data flows involve
multiple end destinations
(1) Boston Consulting Group market study, June 2023; (2) Data input sources and data output sources are platforms that Alteryx can connect to ingest data from (e.g., Excel, Snowflake, Salesforce), or destinations that
Alteryx can output data to (e.g., Tableau, PowerBI) respectively.
-
-
CONFIDENTIAL | 18
Alteryx Designer Product – Use Case Example
Process without Alteryx Designer Process with Alteryx Designer
1. Manually download data from various
systems
2. Import data into multiple Excel sheets
3. Scrub and clean the data (e.g.,
remove extra spaces, split cell
contents)
4. Combine data across sheets
5. Perform manual calculations
6. Create output sheet with tables,
charts, etc.
▪ Import CRM Data
− Create connections to CRM software to extract the latest sales pipeline
▪ Connect to Data Repository
− Pull in historical S&M data stored in data repositories to compare with current pipeline
− Automate process of storing data back to the repository
▪ Prepare
− Create repeatable workflows that clean, filter, and sort the data
▪ Blend
− Combine current pipeline from CRM with historical data to create summary metrics and
measure against past performance
▪ Output
− Browse data in Alteryx to pull high level insights or review the output
− Export data directly to dashboards like Tableau that refresh periodically (e.g., every week),
while simultaneously automating email reports that send daily
1
Manually repeat the above steps as
data refreshes
2
3
4
5
1
2
4
5
3
Note: Processes reflected herein reflect simplified versions of such processes.
-
-
----
-
Connect Select Summarize Analyze
Output
-
to
Data
Data
Data
Data
Data
----------
----
1
R R
-
Blend
-
-
Data
-
-
Connect
Sort
and
Report
Browse
to
Data
Filter Data on
Data
Data
-
D
1
-
CONFIDENTIAL | 19
Customer Case Studies
Reducing Fuel Expenses Streamline Assortment Planning
Situation
• Southwest offered services in 97 destinations with an
average of 3,500 flights a day and wanted to forecast
fuel consumption for each airport
• Initially, the Company used ~150 Excel spreadsheets,
which were disorganized and would take up to three
days to forecast fuel consumption
• With ~$76 billion revenue, ~300,000 employees, and
2,200 stores, assortment planning requires an analysis of
~169,000 distinct SKUs
• Home Depot had spent millions on custom systems to
tackle assortment planning, but could only effectively
analyze ~5% of SKUs given the number of permutations
Alteryx
Solution
• Aggregated monthly data from dozens of sources and
data types
• Filtered and blended the input data
• Used integration with the R coding language to conduct
time series forecasting and build statistical models
• An analyst downloaded an Alteryx trial license
• Within a month, the analyst was able to build workflows to
ingest all the required data to analyze
Results
• Reduced execution time by 99%+ from three days to ten
minutes
• Improved forecasting accuracy by 70% for non-schedule
flights and 12% for schedule flights, resulting in
considerable fuel cost savings
• Once the workflows were set-up, the monthly process
became automated
• Reduced process time by 99%+ from 2 weeks to 30
minutes
• Drove 2-4% increase in sales in the first six months of
deploying Alteryx for merchandise optimization
• Analysis now automatically runs ~10 times per day,
covering 100% of SKUs
Source: Boston Consulting Group.
Note: For illustrative purposes only. Past performance is not indicative of future results and there is no guarantee that the other customers or future customers of the Company have or will have the same or similar results.
Southwest's
THE
HOME
DEPOT
CONFIDENTIAL | 20
Data Market Ecosystem
Ingest data from
key operational
systems and
applications across
the company into a
central analytical
store
Store data in
persistence layer
for analytical use
cases and
computation,
typically in data
lake / warehouse
architectures
Organize data for
use by end-
consumers,
including master
data mgmt.,
quality, and
compliance
Share and
present insights
and findings
across teams,
schedule regular
report updates
and create alerts
Predictive and
prescriptive
analytics
leveraging AI/ML
technologies to
build intelligent
models and
automation
Descriptive
statistics to
extract insights
and trends to
inform decisions
and strategy
Leverage
analytics
expertise and
data to create
embedded BI
solution within
internal or
external products
Funnel insights
and outputs into
key company
applications
enabling
automation of
workflows
Data
Integration
Data
Management
Catalog /
Governance Collaboration
Data
Analytics
(e.g., AI / ML)
Visualization
Create
applications
and embed
Process /
Automation
Data Insights Action
Description
Industry
Players
Designer Server Designer
Intelligence suite
Designer Cloud Machine Learning
Server
Alteryx's
products Designer Cloud
Data
Preparation
Product
Positioning Average No solution Highly
Differentiated
Somewhat
Differentiated Average Below
Average
Below
Average
Below
Average
Prepare and
clean data for use
by business
analysts and data
scientists to
analyze and
generate insights
Somewhat Differentiated
Below Average
Designer
Highly Differentiated
Average
No solution
Analytics Cloud
Auto Insights
No solution
Source: Boston Consulting Group market study, June 2023.
........
......
Fivetran CLOUDERA
Informatica Power
BI
talend
X DataRobot
Ui
Path' Qlik
Q
Qlik Q Qlik Q Ui Path
Informatica snowflake +++
PEGA
Collibra tableau
Ssas
tableau (11] Power
BI
Power B
H20.al
snowflake talend
python
NICE
IBM
Power
BI
SQL
Server databricks
ORACLE
DAlation
data
H20.al
tableau
Iku
alteryx
CONFIDENTIAL | 21
Differentiation from Data Management and ETL
Data Management Extract, Transform, Load (ETL)
Target User• Business analysts• Data engineers• Data engineers
Buying
Organization
• Business units (finance,
operations, etc.)• IT, engineering, data org, etc.• IT, engineering, data org, etc.
Barrier to
Adoption
• Low – tool is utilized by
business analysts without
programing background
• High– requires technical
knowledge
• High– requires technical
knowledge
Data Needs• Typically smaller scale datasets
(typically Gigabytes of data)
• Large datasets (Petabytes of
data)
• Large datasets (Petabytes+ of
data)
“Informatica and Talend's data prep tools are powerful, but they are
clunky, old, have a very high learning curve, and are catered to data
engineers and scientists”
“Data Management players' core competency isn't user-friendly
interface applications”
“Alteryx is quite sticky; strong
adoption is led by the nature
of the product being easy to
use and the shallow
learning curve”
Source: Bain and Boston Consulting Group market studies. The views of other customers may differ.
alteryx snowflake Informatica
alend
databricks
CONFIDENTIAL | 22
Partnership Ecosystem
Solution Providers Global System Integrators OEMs
Technology Partners
Partner ecosystem with 80+ data connectors(1) to a range of data sources
(1) Boston Consulting Group market study, June 2023.
Capitalize. Data Meaning
pwc
aws
THOMSON
REUTERS
protiviti· PRESIDIO.
>
Information Lab
O BILLIGENCE accenture snowflake
tx
+ableau
corptax
CONVERGE keyrus Infosys Ui Path
REPLY
databricks
Almpoint experian. Degatos
NTT DaTa KPMG
MIP
THEOBALD O
SOFTWARE
Seek Velocity slalom ORACLE'
Intalytics
A
Kalibrate
Company
Anordica DATA PREP
U
SAP CASABASE
EY
Sea Cliff ACCELYTICS
CTD SYNNEX
O SHI
HCLTech
A
Microsoft Buxton®
DVW ANALYTICS
Project
Hideaway
Investment Highlights
CONFIDENTIAL | 24
Investment Highlights
1 Leading Player in Large and Growing Market
▪ ~$39B TAM growing at ~12% per annum(1) as Alteryx is a leader in the data prep and
blend market for non-technical users(2)
2 Revenue at Scale
▪ Approaching $1B of ARR growing mid-teens with historical loss-only retention of 95%+
and 2023E net retention of ~112% with limited customer concentration
3 Core Software Solutions
▪ Core data prep-and-blend solution embedded in operational workflows and utilized
across functional groups with 80+ data connectors to all critical data sources
4 Diversified Customer Base
▪ Product leadership (20+ year incumbent) has resulted in a customer base of 49% of
the Global 2000(3) with 8,000+ total customers
5 Opportunity to Expand Profitability
▪ Software benchmarking points to opportunity to optimize costs leveraging Clearlake’s
O.P.S.® framework and Insight’s operational best practices – detailed cost plan ready
to be implemented on day one
6 Platform for M&A
▪ Opportunity to leverage combined Clearlake and Insight resources to pursue accretive
and complementary targets
(1) Boston Consulting Group market study, June 2023; (2) Leading player in data preparation and analytics per Bain market study, November 2023; (3) Alteryx Investor Presentation, November 2023.
Note: Past performance is not indicative of future results and there can be no assurance that any of the trends described above or any other trends will continue or materialize, and there can be no assurance that any assumption forming a part of the estimates or
projections contained herein will continue to be true. Please see Disclaimers herein for important information on estimates, projections and forward looking statements.
alteryx
CONFIDENTIAL | 25
1 Leading Player in Large and Growing Market
$39B
$44B
$50B
$55B
2023 2024 2025 2026
$16B
$18B
$20B
$22B
2023 2024 2025 2026
Total Addressable Market (TAM) Serviceable Addressable Market (SAM)
BCG market work points to a large and growing market given the rapidly growing importance of
data-driven decision making across industries and across functional business units(1)
Note: Alteryx is a leading player in data preparation and analytics per Bain market study, November 2023; (1) Boston Consulting Group market study, June 2023. Past performance is not indicative of future results and
there can be no assurance that any of the trends described above or any other trends will continue or materialize, and there can be no assurance that any assumption forming a part of the estimates or projections
contained herein will continue to be true. Please see Disclaimers herein for important information on estimates, projections and forward looking statements.
CAGR: 12%
CAGR: 12%
CONFIDENTIAL | 26
Feature Richness
User Interface
User Community
Accessibility
Ease of Integration
Pricing
1 Leading Player in Large and Growing Market
Feature richness & user interface has led Alteryx to be considered well regarded amongst its core
competitors(1)
Note: Alteryx is a leading player in data preparation and analytics per Bain market study, November 2023; (1) Boston Consulting Group market study, June 2023. Past performance is not indicative of future results. There
are no guarantees the Company will achieve similar results as its competitors and other competitors not included in this Presentation may also serve as relevant comparison companies. There are significant differences
between the Company and the other competitors shown herein. The Company may have a greater degree of risk and volatility than investments by the other peers shown herein. It should not be assumed that there will be
a correlation between the Company's performance or valuation.
alteryx data
Open
for Innovation
iku
KNIME tableau
Power
B
- -
--
--
-
CONFIDENTIAL | 27
$373
$493
$638
$834
$948
2019A 2020A 2021A 2022A 2023E
2 Revenue at Scale
ARR
($ in millions)
ARR scale growing historically at a 25%+ CAGR
Source: Company financials and forecast.
Note: Past performance is not indicative of future results and there can be no assurance that any of the trends described above or any other trends will continue or materialize, and there can be no assurance that any
assumption forming a part of the estimates or projections contained herein will continue to be true. Please see Disclaimers herein for important information on estimates, projections and forward looking statements.
CAGR: 26%
1
CONFIDENTIAL | 28
2 Revenue at Scale
ARR scale anchored by retention profile
When excluding smaller customers (~10% of revenue), gross retention exceeds 95% with loss-only retention close to 100%
~92% ~91% ~90%
~123%
~116%
~112%
2022A LTM Q3'23 2023E (1)
Gross Retention (incl. Downsell) Net Retention
Gross and Net Retention
Uncertain macroeconomic
environment contributing to temporary
slowdown in expansion causing
decline in Net Retention Rate
Loss-Only Retention: ~97% Loss-Only Retention: ~96%
Source: Historical retention analysis per KPMG.
(1) Internal management budget implies 90%+ gross retention. Clearlake and Insight case assumes slightly more churn / downsell in 2H’23.
Note: Past performance is not indicative of future results and there can be no assurance that any of the trends described above or any other trends will continue or materialize, and there can be no assurance that any
assumption forming a part of the estimates or projections contained herein will continue to be true. Please see Disclaimers herein for important information on estimates, projections and forward looking statements.
CONFIDENTIAL | 29
Chief Commercial
Officer
Chief Financial
Officer
Chief Marketing
Officer
Chief Operating
Officer
Chief Revenue
Officer
Chief Information /
Data Officer
Customer
Analytics, Data
Science &
Processes
Finance, G&A
Analytics, Data
Science &
Processes
Marketing
Analytics, Data
Science &
Processes
Operations, Supply
Chain, Network
Analytics & Data
Science
Sales, Revenue,
Channel Analytics,
Data Science &
Processes
IT & Data
Infrastructure
Analytics &
Processes
Consumer
Behavior
Trading &
Liquidity
Marketing
Attribution
Logistics
Optimization Sales Trends Network Analysis
Costumer
Engagement
Capital
Forecasting
Predictive
Conversion
Labor
Optimization
Sales
Maximization
Security
Analytics
Pricing
Optimization
Daily
Reconciliation
Campaign
Response
Cost
Optimization
Sales
Performance Cost Analysis
Sentiment
Analysis Tax Automation Segmentation Capacity
Planning Forecasting CapEx
Forecasting
3 Core Software Solutions
Data Analytics Influences the Entire C-Suite
High Quality Data is Critical to Key Business Functions
Alteryx is utilized by teams across functional groups for a variety of use cases
Source: Bain and Boston Consulting Group market studies.
CONFIDENTIAL | 30
3 Core Software Solutions
Likelihood of Renewal Difficulty in Switching
Customer feedback points to switching costs due to how embedded Alteryx is within
workflows, driving likelihood of renewal
Alteryx is so penetrated in our ecosystem, if we had to replace it with
something else, we would need to plan that 2 years ahead…
The back-end for all of our business intelligence landscape is built off
pipelines in Alteryx…switching would mean that our BI / analytics would
grind to a halt for an entire month
5%
6%
29%
59%
Users
Unlikely Neutral Likely Highly Likely
N = 164
3%
68%
21%
9%
Users
Not Difficult Difficult Very Difficult Extremely Difficult
N = 116
Source: Boston Consulting Group. The views of other customers may differ.
CONFIDENTIAL | 31
3 Core Software Solutions
90% 91%
86% 85%
77%
81%
76% 75%
80% 77%
83%
78%
Q1-22 Q2-22 Q3-22 Q4-22 Q1-23 Q2-23
% Activated Seats / Seats Sold (1) % Active Users / Activated Seats (2)
Utilization rates illustrate how embedded Alteryx is in its customer’s day-to-day workflows
Product Utilization Data
Organizations typically expand their seat licenses over time as they identify new use cases –
utilization consistently above ~75%
Source: Bain and Boston Consulting Group market studies. (1) Represents the % of seats sold in a quarter that were then activated (i.e., seats that are implemented, turned on, and ready to use). (2) Represents the % of activated seats that were active in the
quarter (i.e., users that have opened and used the Alteryx product).
Note: Past performance is not indicative of future results and there can be no assurance that any of the trends described above or any other trends will continue or materialize. Please see Disclaimers herein for important information on estimates, projections
and forward looking statements.
CONFIDENTIAL | 32
4 Diversified Customer Base
Retail Food & Services Consumer Telecom Media Prof. Services
Energy & Utilities Healthcare Manufacturing Financial Services Travel & Hospitality Technology
8,000+ Customers with 49% of the Global 2000
Source: Bain and Boston Consulting Group market studies.
Stanley
Walgreens AutoNation KraftHeinz OXXO NETFLIX
Black &
Decker
HAKUHODO -----
NTT
swisscom WARNER
MUSIC GROUP
ROSS Kellogg's Campbelli Whirlpool P&G
Bell BT o\
Audacy UNIVERSAL BAIN
MOL
amazon services &
COMPANY
????
DRESS FOR LESS
WORKWEAR M&S Kroger
Nexstar
SHISEIDO
telenor
KDDI --- LIONSGATE
SIQVIA HCL
Nestlé
Kingfisher NORDSTROM coles
ESTEE |FOXy IMG
DANONE CGI SKANSKA LAUDER
BAYER
Claro-
vodafone
Chevron
TEPCO Abbott Stanford gm VISA BMG PALMS Radisson 00 Meta salesforce
HEALTH CARE
TEPCO Energy Partner HYUNDAI CASIO/RESORT/SPA
Cedars MARS DAIKIN
Marriott
ANA
DELL
NEC
Sinai
PETROBRAS novo nordisk
JBIC
INTERNATIONAL
Sempra Energy INTERNATIONAL
NORTHROP GRUMMAN
SOCIETE
oxpodia
AVIS
ENBRIDGE Queensland Government
Itaú
IBM
GENERALE e
group
Hewlett
Packard
Enterprise
soudi Gromco MSIG
PAPER
Allianz (Ili SUNCORPO
Santander TOLL
McAfee
World Fuel SIEMENS
LUZ DEL SUR
CONFIDENTIAL | 33
4 Diversified Customer Base
Increasing Focus on Enterprise Low Customer Concentration (% ARR)
Top 10 Customers Other
Top 10 Accounts
9% of ARR
467 ~970
~2x
<$50K ~$109K
~2x
19 166
~9x
2018 Current
Customers with
$1M+ in ARR
ARR
per Customer
Global G2K
Customers
Source: Company financials.
Note: Past performance is not indicative of future results and there can be no assurance that any of the trends described above or any other trends will continue or materialize.
C
CONFIDENTIAL | 34
5 Opportunity to Expand Profitability
P&L Focus Areas
COGS▪ Low penetration of offshoring (9%) presents opportunity to offshore additional non-customer-funded headcount
▪ Additional cost savings opportunities through the reduction of overhead roles (e.g., Office of CCO)
S&M
▪ Low spans of control (4.1:1) and overallocation of sales support functions including sales engineers, sales
operations, channel, etc.
▪ Marketing spend is focused on employee branding and concentrated in expensive sponsorships (e.g., McLaren)
and an annual customer event ($30M+)
▪ Presents opportunity to:
i. Shift go-to-market motion to land-and-expand and reps to hunter-farmer
ii. Right-size account executive capacity, management layers and sales support functions
iii. Reduce marketing spend and team size
R&D
▪ R&D cost reduction opportunities include:
i. Continued focus on offshoring (currently 28% of headcount)
ii. Improve spans of control and reduced management positions
iii. Improve non-headcount overhead costs including internal test beds
G&A
▪ G&A cost savings will be focused on:
i. Removal of public company resources and costs
ii. Reduction of back-office complexity and improved spans of control
iii. Offshoring additional positions across back-office functions
Despite having ~90% gross margins, the company has historically operated with a “growth at all
costs” mindset leaving substantial opportunity to right size its OpEx spend
Source: Mix of Company, Clearlake, and Insight historical analysis and projections.
Note: Past performance is not indicative of future results and there can be no assurance that that the initiatives described above will be implemented, or if implemented, that they will be successful. There can be no assurance that any projections will be achieved. Please see Disclaimers herein for important
information on projections and forward looking statements, as well as financial metrics used herein. Statements made herein may represent opinions and beliefs of the Insight, Clearlake or the Company and should not be relied upon as a promise or representation as to past or future performance. Forward looking
statements contained herein are only estimates of future results or events that are based upon assumptions made at the time such statements were developed or made.
-
CONFIDENTIAL | 35
5 Opportunity to Expand Profitability
P&L HC Impact
(FTEs)
HC Impact
(%)
HC Savings
($M)
Non-HC Savings
($M)
Total Savings
($M)
Savings
(%)
COGS 45 16% $8 - $8 8%
S&M 470 37% $117 $30 $147 30%
R&D 130 19% $18 $5 $23 15%
G&A 140 35% $30 $10 $40 38%
Total 785 30% $173 $45 $218 26%
Visibility to ~$218M of cost cuts within 18 months of entry representing opportunity to drive growth at
40%+ long-term margins
Source: Mix of Company, Clearlake, and Insight projections.
Note: Past performance is not indicative of future results and there can be no assurance that that the initiatives described above will be implemented, or if implemented, that they will be successful. There can be no assurance that any projections will be
achieved. Please see Disclaimers herein for important information on projections and forward looking statements, as well as financial metrics used herein. Statements made herein may represent opinions and beliefs of the Insight, Clearlake or the Company
and should not be relied upon as a promise or representation as to past or future performance. Forward looking statements contained herein are only estimates of future results or events that are based upon assumptions made at the time such statements
were developed or made.
-
CONFIDENTIAL | 36
5 Opportunity to Expand Profitability
$96M
$152M
$484M
$104M
COGS (Excl. Royalties) R&D S&M G&A
Cost base as of 12/31/23E run-rate – opportunity for further cost optimization. Current cost plan of
$218M represents ~26% of run-rate cost base
Source: Company financials.
Note: Past performance is not indicative of future results and there can be no assurance that that the initiatives described above will be implemented, or if implemented, that they will be successful. There can be no assurance that any projections will be
achieved. Please see Disclaimers herein for important information on projections and forward looking statements, as well as financial metrics used herein. Statements made herein may represent opinions and beliefs of the Insight, Clearlake or the Company
and should not be relied upon as a promise or representation as to past or future performance. Forward looking statements contained herein are only estimates of future results or events that are based upon assumptions made at the time such statements
were developed or made.
CONFIDENTIAL | 37
• Acquisition of Alteryx
ANZ (~$6M deal value
in 2018), exclusive
master distributor for
Alteryx products since
2015
• Acquisition of
ClearStory Data
(~$20M deal value in
2019), an enterprise-
scale continuous
intelligence analytics
solution for
unstructured data
• Acquisition of
Feature Labs
(~$25M deal value in
‘19); platform builds
tools and APIs for
data science
• Acquisition of Lore
IO (~$10M deal
value in 2021), a
no-code data
management
platform that is now
Alteryx’s Machine
Learning capability
• Acquisition of
Hyper Anna
(~$25M deal value
in 2021), a platform
that discovers
insights
automatically using
existing workflows
that is now Alteryx’s
Auto Insights
capability
• Expand footprint into
Australia and New
Zealand regions with
100+ current users
(2018), and help
expand and support the
market
• ClearStory:
Leverage specialty in
intelligent automation
for data processing
to create a natural
extension to Alteryx
Technology Center
• Feature Labs: To
advance machine
learning abilities to
create insights for
users
• Enhance cloud-
native, elastic
computing within
Alteryx Designer
Cloud and Alteryx
Machine Learning,
allowing customers
to analyze large
datasets for
actionable self-
service insights
• Acquisition of Yhat
(~$11M deal value in
2017), a data science
platform for
developing, deploying,
and managing real-
time decision APIs
• Acquisition of
Semanta, a company
specializing in meta
data management and
governance solutions
• Yhat: Enhance data
analytics platform,
including rapidly
deployed advanced
analytical models
• Semanta: Build Alteryx
Connect, allowing
clients to capture data
across their org and
ensure analysts have
the best sources of
information
• Strengthen data
analysis
capabilities by
integrating into
Alteryx tools to
provide insights on
one platform
Description
Strategic
rationale
• Acquisition of
Trifacta (~$400M
deal value in 2022),
a collaborative data
engineering cloud
platform to
transform data,
ensure quality, and
automate pipelines
• Build Designer
cloud platform for
data preparation
and engineering,
cloud reporting and
analytics, and cloud
data onboarding
• Anchors and
accelerates journey
to the cloud and
open new
categories of
buyers across IT
within enterprises
2022
2021
2021
2019
2018
2017
6 Platform for M&A
Alteryx has experience making acquisitions
Source: Company website, Press reviews, Crunchbase.
Note: Past performance is not indicative of future results.
yhat a ClearStory
Lore
IO
HYPER ANNA
FEATURE LABS
TRIFACTA
CONFIDENTIAL | 38
6 Platform for M&A
Opportunities to augment product suite and expand into new parts of the value chain
D
a
t
a
S
c
ie
n
c
e
P
l a
t f
o
r
m
s
D
a
t
a
G
o
v
e r n
a
n c e
&
S
e
c
u
r
i
t
y
D
a
t
a
M
a
n
a g e m e n t
P
l
a
t
f
o
r
m
s
M
e
t
a
D
a
t
a
M
a
n
a
g
e
m
e
n
t
(
M
D
M
)
A
I
A
n
a l y t i c s
P l a t f o r
m
s
A d d i t i o n a l S e r v i c e s ( e.
g.
,
L
o
c
a
t
i
o
n
D a t a )
▪ Revenue and cost synergies with
adjacent market vendors
▪ Expand value proposition through
additional solutions (e.g., MDM)
▪ Augment AI/ML and visualization
capabilities
▪ Increase retention by building out data
governance capabilities
▪ Large opportunity to cross-sell and up-
sell incremental data tools
Source: Mix of Clearlake and Insight projections.
Note: Past performance is not indicative of future results and there can be no assurance that that the initiatives described above will be implemented, or if implemented, that they will be successful. There can be no assurance that any projections will be
achieved. Please see Disclaimers herein for important information on projections and forward looking statements, as well as financial metrics used herein. Statements made herein may represent opinions and beliefs of the Insight, Clearlake or the Company
and should not be relied upon as a promise or representation as to past or future performance. Forward looking statements contained herein are only estimates of future results or events that are based upon assumptions made at the time such statements
were developed or made.
Data
Governance
& Security
Data Science
Platforms
ANACONDA
privacera
rapidminer IMMUTA"
Data
Management Platforms
AKNIME
OKËRA
DAlation
data
1
spatial O
mapbox
iku
C) sisense
Collibra
TI. ThoughtSpot
maptiler CARTO
@geopointe
Syniti
denodo
***
FOURSQUARE
Semarchy
Data)
DOMO
H20,ai
Glympse® ui Path® :Profisee
Airtable
scale
.tamr
RELTIO
Additional
Services
(e.g .,
Location
DataRobot
/~
matillion
Meta
Data
Management
(MDM)
d GoodData
Al
Analytics
Platforms
Project
Hideaway
Financial Overview
CONFIDENTIAL | 40
Financial Highlights
~$948M ARR ~14% ARR Growth
~112% Net Retention ~90% Gross Retention(1) /
~96% Loss-Only Retention(2)
~38% PF Adj. Cash EBITDA
Margin
~90% Gross Margin
(1) All financial metrics are for FY2023E, pro forma for $218M in cost savings. (1) includes downsell (2) Loss-Only as of LTM 09/30/23. Mix of Company, Clearlake, and Insight estimates.
Note: Past performance is not indicative of future results and there can be no assurance that any of the trends described above or any other trends will continue or materialize.
CONFIDENTIAL | 41
Topline Metrics
$536
$855
$958
2021A 2022A 2023E
$638
$834
$948
2021A 2022A 2023E
($ in millions) ($ in millions)
ARR GAAP Revenue
Source: Company financials. Includes Trifacta acquisition in 2022A (~$25M of ARR).
Note: Past performance is not indicative of future results and there can be no assurance that any of the trends described above or any other trends will continue or materialize, and there can be no assurance that any
assumption forming a part of the estimates or projections contained herein will continue to be true. Please see Disclaimers herein for important information on estimates, projections and forward looking statements.
CAGR: 22%
CAGR:
34%
CONFIDENTIAL | 42
Retention Rates
When excluding smaller customers (~10% of revenue), gross retention exceeds 95% with loss-only retention close to 100%
~92% ~91% ~90%
~123%
~116%
~112%
2022A LTM Q3'23 2023E (1)
Gross Retention (incl. Downsell) Net Retention
Uncertain macroeconomic
environment contributing to temporary
slowdown in expansion causing
decline in Net Retention Rate
Loss-Only Retention: ~97% Loss-Only Retention: ~96%
Gross and Net Retention
Source: Historical retention analysis per KPMG.
(1) Internal management budget implies 90%+ gross retention. Clearlake and Insight case assumes slightly more churn / downsell in 2H’23.
Note: Past performance is not indicative of future results and there can be no assurance that any of the trends described above or any other trends will continue or materialize. Please see Disclaimers herein for important information on estimates, projections
and forward looking statements.
CONFIDENTIAL | 43
Retention Metrics by Customer Size
Retention Metrics (Customers with $50K+ ARR)
~97% ~96% ~96%
~100% ~100% ~100%
~128% ~129%
~123%
2021A 2022A LTM Q3'23A
Gross Retention (incl. Downsell) Loss-Only Retention Net Retention
Gross and net retention higher for core customer base with $50K+ in ARR (~90% of total ARR)
Source: Historical retention analysis per KPMG.
Note: Past performance is not indicative of future results and there can be no assurance that any of the trends described above or any other trends will continue or materialize.
CONFIDENTIAL | 44
Alteryx has Grown Share with Larger Enterprises
ARR per Customer vs. Number of Customers with $1M+ in ARR
$71 $74 $75
$80 $83 $88 $91
$100 $103 $106 $109
66 70
83 91 99
111 119
142 142
152
166
–
$20
$40
$60
$80
$100
$120
$140
$160
Q1-21 Q2-21 Q3-21 Q4-21 Q1-22 Q2-22 Q3-22 Q4-22 Q1-23 Q2-23 Q3-23
ARR per Customer Number of Customers with $1M+ in ARR
Increase in ARR per customer and customers with $1M+ in ARR highlights the value proposition of
the Alteryx platform and its increase in customers’ share of wallet
Source: Company financials.
Note: Past performance is not indicative of future results and there can be no assurance that any of the trends described above or any other trends will continue or materialize.
CONFIDENTIAL | 45
$26
$196
$25 $52 $18
$183
$88
ARR Bridge
$638
$834
$948
Trifacta 2023E
EOP (1)
Expansion Downsell
+ Churn
Expansion
New Logo New Logo
2021A
EOP
2022A
EOP
Downsell
+ Churn
2022A 2023E
ARR Bridge
($ in millions)
(1) Internal management budget implies $960M of ARR. Clearlake and Insight case conservatively assumes 2023E ending ARR in-line with management’s public guidance from 3Q23 earnings.
Note: Past performance is not indicative of future results and there can be no assurance that any of the trends described above or any other trends will continue or materialize.
CONFIDENTIAL | 46
$492
$768
$855
$8
$863
92% 90% 90%
2021A 2022A PF 2023E
PF Cost Savings
($ in millions) ($ in millions)
90%+ Gross Margins with Future Operating Leverage
PF Gross Profit PF Adj. Cash EBITDA
$146
$218
$109 $101
$364
20% 12%
38%
2021A 2022A PF 2023E
PF Cost Savings
Source: Company financials. PF 2023E Gross profit and PF 2023E Adj. Cash EBITDA includes $218M of PF cost savings. Cost savings expected to be actioned within 6-18 months. 2021A and 2022A represent standalone (excl. PF cost savings).
Note: Past performance is not indicative of future results and there can be no assurance that any of the trends described above or any other trends will continue or materialize.
CONFIDENTIAL | 47
Key Projection Assumptions: Base Case
Top Line
▪ Assumed slower growth in 2024E driven by cost-out and GTM reorganization, followed by high-single-digit
growth through 2029E
▪ Gross dollar retention (including down-sells) modeled in low 90s with net dollar retention stable at 106% given
heightened focus on health of customer base and re-organized go-to-market strategy (customer success /
renewals managers untouched by cost restructure plan)
Expenses
▪ Slight gross margin decline as mix of lower-gross margin cloud solutions increases
▪ $218M of pro forma cost cuts with expected completion over first 18 months
o Assumed $90M of cash costs for severance, offshoring initiatives, etc. to achieve contemplated synergies
▪ Operating leverage during projection period driven by implementation of Clearlake and Insight best practices,
with PF Adj. EBITDA expanding to ~40% – in-line with other software companies of a similar size and growth
profile
Taxes▪ Tax rate of 26%
▪ Utilization of $128M+ existing NOL balance results in minimal cash taxes in early years
Working
Capital
▪ Working capital (excluding deferred revenue) conservatively assumed to be an outflow of cash
▪ Positive change in deferred revenue driven by continued growth in the projection period
Other Cash
Flow Items
▪ Limited capital expenditures in the projection period (<5% historically)
▪ No mandatory amortization in first two years
Note: These assumptions are based on the Sponsors’ experience and may not prove to be accurate. Variations from assumptions may negatively affect the performance of the investment.
CONFIDENTIAL | 48
Illustrative Equity Returns
Base Case yields ~2.6x MOIC with multiple organic and inorganic levers to drive further equity
value creation over 4-year hold
Commentary Gross
MOIC / IRR
Net 1
MOIC / IRR
Base
Case
▪ Asset of scale with improved balance between profitability and growth
increases attractiveness to potential strategic acquirors and financial sponsors
(or for a public relisting)
▪ Revenue CAGR: ~7% | Target PF Adj. Cash EBITDA Margin: ~46% | Exit NTM
PF Adj. Cash EBITDA-CapSW Multiple: 15.5x
2.6x / 27% 2.5x / 26%
Base
Case +
M&A
▪ Highly-centralized workflow and status as a platform opens doors to acquire
additional strategic solutions at attractive PF Cash EBITDA margins
▪ Assumes ~$40M of acquired PF Cash EBITDA (50% PF margin) annually at a
~8x PF EBITDA multiple – in-line with historical M&A execution at other
Clearlake and Insight portfolio companies
▪ Acquisitions are financed with additional debt and $40M of additional equity
(1.0x PF EBITDA)
3.0x / 32% 2.9x / 32%
(1) Net projections reflect up to 3% of LP original commitments to pay partnership expenses.
Note: The projections provided herein are not reflective of actual performance and are hypothetical. These hypothetical projections are based on Clearlake and Insight analysis, estimates and projections. There can be no assurance that the assumptions
made or the beliefs and expectations currently held by Clearlake and Insight will prove correct and actual events and circumstances may vary significantly. Statements contained herein are based on current expectations, estimates, projections, opinions
and/or beliefs of Clearlake and Insight. Such statements involve known and unknown risks, uncertainties and other factors, and undue reliance should not be placed thereon. No representation is made or assurance given that such statements or views are
correct. There can be no assurance that any projections will be achieved. Please see Disclaimers for important information regarding these projections.
CONFIDENTIAL | 49
Summary Organic P&L Forecast: Base Case
Organic P&L Historical Mgmt. CCG / Insight Forecast - Annual CAGR
($ in millions; FYE 12/31) 2022A 2023E 2024E 2025E 2026E 2027E 2028E 2029E '23-'29
BoP ARR $638 $834 $948 $1,013 $1,085 $1,169 $1,259 $1,360
(+) New Logo 26 18 18 19 20 21 22 24
(+) Expansion 196 183 161 165 172 181 191 203
(-) Churn / Downsell (52) (88) (114) (111) (109) (111) (113) (116)
(+) Trifacta Acquisition 25– – – – – – –
EoP ARR $834 $948 $1,013 $1,085 $1,169 $1,259 $1,360 $1,471 8%
% yoy growth 31% 14% 7% 7% 8% 8% 8% 8%
Gross $ Retention 92% 90% 88% 89% 90% 91% 91% 92%
Net $ Retention 123% 112% 105% 105% 106% 106% 106% 106%
GAAP Revenue $855 $958 $980 $1,049 $1,127 $1,214 $1,309 $1,415 7%
% yoy growth - organic 60% 12% 2% 7% 7% 8% 8% 8%
PF Gross Profit $776 $863 $867 $925 $991 $1,063 $1,143 $1,231 6%
% margin 91% 90% 89% 88% 88% 88% 87% 87%
PF Adj. EBITDA $255 $336 $370 $404 $450 $503 $565 $627 11%
% margin 30% 35% 38% 39% 40% 41% 43% 44%
(+) Change In Deferred Revenue 64 28 12 16 21 22 23 24
PF Adj. Cash EBITDA $319 $364 $382 $420 $472 $525 $588 $651 10%
% margin 37% 38% 39% 40% 42% 43% 45% 46%
Source: Mix of Company, Clearlake, and Insight estimates.
Note: Past performance is not indicative of future results and there can be no assurance that any of the trends described above or any other trends will continue or materialize, and there can be no assurance that any assumption forming a part of the estimates or projections contained herein will continue to be
true. While such assumptions are deemed reasonable by Insight and Clearlake under the current circumstances, a number of risks, market events and other similar factors could result in such assumptions no longer being accurate. Any changes in the applicability of key assumptions may have a material effect
on the estimates and projections contained herein. In addition, while Insight believes these considerations are material, they are not an exhaustive list of considerations that may be material to the Company and there may be other considerations that are not included herein which may have a material adverse
impact on the Company. Statements made herein may represent opinions and beliefs of the Sponsors, and should not be relied upon as a promise or representation as to past or future performance. Please see Disclaimers herein for important information on estimates, projections and forward looking
statements. Certain information contained herein has been obtained from third parties, including Company management. While such information is believed to be reliable, such information may be based on assumptions that later prove to be invalid or incorrect and is not provided with a view to providing
investment advice with respect to any security, or making any claim as to the past, current or future performance thereof.
CONFIDENTIAL | 50
Projected Organic Summary Financials: Base Case
Gross & Net Retention
PF Adj. Cash EBITDA
ARR
Free Cash Flow(1)
88% 89% 90% 91% 91% 92%
105% 105% 106% 106% 106% 106%
2024 2025 2026 2027 2028 2029
Gross Retention (incl. Downsell) Net Retention
$1,013 $1,085 $1,169 $1,259 $1,360 $1,471
7% 7% 8% 8% 8% 8%
2024 2025 2026 2027 2028 2029
ARR % Growth
$382 $420 $472 $525 $588 $651
39% 40% 42% 43% 45% 46%
2024 2025 2026 2027 2028 2029
PF Adj. Cash EBITDA % Margin
$372 $410 $460 $513 $575 $636
97% 98% 98% 98% 98% 98%
2024 2025 2026 2027 2028 2029
FCF (1) % Conversion
($ in millions) ($ in millions)
($ in millions)
Source: Mix of Company, Clearlake, and Insight estimates.
(1) Defined as PF Adj. Cash EBITDA less CapEx.
Note: Past performance is not indicative of future results and there can be no assurance that any of the trends described above or any other trends will continue or materialize, and there can be no assurance that any assumption forming a part of the estimates or projections contained herein will continue to be true.
While such assumptions are deemed reasonable by Insight and Clearlake under the current circumstances, a number of risks, market events and other similar factors could result in such assumptions no longer being accurate. Any changes in the applicability of key assumptions may have a material effect on the
estimates and projections contained herein. In addition, while Insight believes these considerations are material, they are not an exhaustive list of considerations that may be material to the Company and there may be other considerations that are not included herein which may have a material adverse impact on
the Company. Statements made herein may represent opinions and beliefs of the Sponsors, and should not be relied upon as a promise or representation as to past or future performance. Please see Disclaimers herein for important information on estimates, projections and forward looking statements. Certain
information contained herein has been obtained from third parties, including Company management. While such information is believed to be reliable, such information may be based on assumptions that later prove to be invalid or incorrect and is not provided with a view to providing investment advice with respect
to any security, or making any claim as to the past, current or future performance thereof.
Project
Hideaway
Q&A
"""

In [10]:
os.getenv("COHERE_API_KEY")

'dHJjkplGyR7UhYaCcs26Wj4LqROFS1wTVrc79Ylt'

In [13]:
co = cohere.Client(
    api_key='Ldp4S4gZ2FUAwZVvoievRf44t4Y8HPFDPsKcQNem',
    base_url="https://stg.api.cohere.com/v1"
)

In [15]:
prompt = GENERATION_PROMPT.format(document=DOCUMENT)

res = co.chat(
    message=prompt,
    model="command-r-plus"
).text

In [17]:
print(res)

## **Investment Proposal**

We seek your approval to invest in a co-investment opportunity alongside Clearlake Capital Group, L.P. ("Clearlake") and Insight Partners ("Insight") to support the take-private acquisition of Alteryx, Inc. ("Alteryx" or the "Company"), a leading provider of data preparation and analytics software for non-technical users. The total enterprise value (TEV) of the transaction is estimated at $4.6 billion, implying a multiple of 13.6x 2023 expected adjusted EBITDA, excluding capitalized software (Adj. Cash EBITDA).

## **Key Stakeholders and Co-Investors**

Clearlake, a leading private investment firm, and Insight, a private equity firm with expertise in technology investments, are co-leading this acquisition. Clearlake has a strong track record in software and technology-enabled services, while Insight brings extensive knowledge and experience in the data analytics space. The co-investors will work together to unlock Alteryx's growth potential and enhance its m